In [1]:
#Framework for performing the experiments for the car data set
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
from sklearn import linear_model, tree, neural_network, preprocessing
import numpy as np
import random
import os
import copy
import math
from matplotlib import pyplot as plt
from datetime import datetime
from itertools import combinations

In [2]:
#Import the data set and remove the rows with missing values
car_data = pd.read_table('datasets/car/car-original.data', header=None, delim_whitespace=True)
df = car_data.apply (pd.to_numeric, errors='coerce')
print(df)
clean_data = df = df.dropna()
clean_data.reset_index(drop=True)
npdata = clean_data.to_numpy()


24


In [3]:
# print(npdata)

In [ ]:
#define the test indices and define the test data. This data remains untouched
random.seed(50)
test_indices = np.unique(np.array([random.randint(0,npdata.shape[0]-1) for k in range(0,math.ceil(0.2*npdata.shape[0]))]))
# print(test_indices)
test_data = npdata[test_indices, :]
test_X = test_data[:,1:]
test_transformer = preprocessing.MinMaxScaler().fit(test_X)
normalized_test_X = test_transformer.transform(test_X)
real_mpg = test_data[:, 0]



#set up iteration variables and parameters
mu = 0
#hp = 1st column, weight = 2nd column, acceleration = 3rd column
features = [[1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]] 
input_variances = np.arange(0,0.4,0.01)
iterations = 100


#set up arrays for the results of the models
results = np.zeros([iterations, test_X.shape[0]])
output_variances = np.zeros(input_variances.size)
plot_matrix = np.zeros([len(features), input_variances.size])

for j in range(0, len(features)):
    k = 0
    for sig in input_variances:
        rng = np.random.default_rng(12345)
        for i in range(0, iterations):
            data = copy.deepcopy(npdata)
            training_data = np.delete(data, test_indices, axis=0)
            rows = training_data.shape[0]
            noise_indices = rng.choice(rows-1, math.ceil(0.1*rows), replace=False)
            training_X = training_data[:, 1:]
            training_transformer = preprocessing.StandardScaler().fit(training_X)
            normalized_training_X = training_transformer.transform(training_X)
            for el in features[j]:
                noises = rng.normal(mu, sig, noise_indices.shape)
                normalized_training_X[noise_indices, el] += noises
            training_y = training_data[:, 0]
            ######################################################### LINEAR REGRESSION
            clf = linear_model.LinearRegression()
            ######################################################### DECISION TREE
            # clf = tree.DecisionTreeRegressor(random_state=42)
            ######################################################### NEURAL NET
            # clf = neural_network.MLPRegressor(random_state=42, max_iter=200)
            #########################################################
            clf = clf.fit(normalized_training_X, training_y)
            prediction_mpg = clf.predict(normalized_test_X)
            results[i] = prediction_mpg
        variances = np.array([np.var(results[:, k]) for k in range(0, results.shape[1])])
        mean_output_variance = np.mean(variances)
        print(mean_output_variance)
        output_variances[k] = mean_output_variance
        plot_matrix[j] = output_variances
        k += 1
# print(output_variances)
print("done")
now = datetime.now()
timestamp = datetime.timestamp(now)
experiment_name = "lin reg extra"
path = 'results/car/'
np.savetxt(path + experiment_name + str(timestamp) + '.csv', plot_matrix, delimiter=",")

1.8688847367953315e-33
1.6853164583771687e-08
6.666826889578352e-08
1.475383123652266e-07
2.5660726116708893e-07
3.9024488791150245e-07
5.442586478615131e-07
7.141226822005417e-07
8.952043452831699e-07
1.0829712933147627e-06
1.2731664252468703e-06
1.4619429656313333e-06
1.645956904943538e-06
1.8224182359332758e-06
1.9891055713226265e-06
2.144350678164101e-06
2.287000298937004e-06
2.4163625591188634e-06
2.532144551528904e-06
2.6343866031555004e-06
2.7233974951186417e-06
2.799693688869379e-06
2.8639445208939123e-06
2.9169244217432828e-06
2.959472511514065e-06
2.992459414303998e-06
3.0167607944849945e-06
3.0332369166304167e-06
3.042717436306073e-06
3.045990611149879e-06
3.0437961555554077e-06
3.036821027903283e-06
3.0256975215183536e-06
3.011003118636523e-06
2.9932616533846596e-06
2.972945410602886e-06
2.950477859815801e-06
2.9262367867345862e-06
2.9005576382566426e-06
2.873736941535185e-06
1.8688847367953315e-33
2.6195545553378962e-08
1.045713141117957e-07
2.3464022065866507e-07
4.156909

In [ ]:

#Plot the outcomes
t=0
for arr in plot_matrix:
    plt.plot(input_variances, arr, label="line " + str(features[t]))
    t+=1
plt.xlabel('input variance')
plt.ylabel('output variance')
plt.legend()
plt.savefig(path + experiment_name + str(timestamp) + '.png')
plt.show()

In [ ]:
now = datetime.now()
timestamp = datetime.timestamp(now)

